In [83]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer,make_column_selector

from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor



from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error,r2_score

### static functions

In [84]:
def prepare_data(df):
    drop_col =[
        "Id",
        "Condition1",
        "Condition2",
        "Exterior1st",
        "Exterior2nd",
        "MiscVal",
        #"3SsnPorch"
        #"MoSold"
        ]
    for col in drop_col:
        if col in df.columns:
            df = df.drop(columns=col)
            
    #Dirty solution to replace "None" with "NA" in one column to be the same as others
    df.loc[df.MasVnrType == "None","MasVnrType"] = "NA"
    
    return df

In [85]:
def find_columns(all_cat_col,which):
    dict_ordinal={
    "Utilities":["NA","ELO","NoSeWa","NoSewr","AllPub"],
    "LandSlope":["NA","Sev","Mod","Gtl"],
    #"HouseStyle":["1Story","1.5Fin","1.5Unf","2Story","2.5Fin","2.5Unf","SFoyer","SLvl"],
    "RoofMatl":["NA","ClyTile","CompShg","Membran","Metal","Roll","Tar&Grv","WdShake","WdShngl"],   #Not sure
    "MasVnrType":["NA","BrkCmn","BrkFace","CBlock","Stone"],
    "Foundation":["NA","BrkTil","CBlock","Slab","PConc","Wood","Stone"], #Not sure
    "BsmtExposure":["NA","No","Mn","Av","Gd"],
    "BsmtFinType1":["NA","Unf","LwQ","Rec","BLQ","ALQ","GLQ"],
    "BsmtFinType2":["NA","Unf","LwQ","Rec","BLQ","ALQ","GLQ"],
    "GarageFinish":["NA","Unf","RFn","Fin"],
    "PavedDrive":["NA","N","P","Y"]
    #"Fence":[]
    }
    cat = []
    no_qu_col=[]
    qu_col=[]
    choose_col=[]
    for col in all_cat_col:
            if (col.endswith("Qual")
            or col.endswith("Qu") 
            or col.endswith("QC") 
            or col.endswith("Cond")):
                qu_col.append(col)
            elif col not in list(dict_ordinal.keys()):
                no_qu_col.append(col)
        
    if which == "quality":
        choose_col = qu_col
        cat=["NA","Po","Fa","TA","Gd","Ex"]

    elif which == "notquality":
        choose_col = no_qu_col
    
    elif which == "ordinal":
        choose_col= list(dict_ordinal.keys())
        cat = list(dict_ordinal.values())
        
    return choose_col,cat

In [95]:
qu_col=[]
for col in list(data_df.columns):
    if (col.endswith("Qual")
        or col.endswith("Qu") 
        or col.endswith("QC") 
        or col.endswith("Cond")):
            qu_col.append(col)


In [96]:
qu_col

['OverallQual',
 'OverallCond',
 'ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'HeatingQC',
 'KitchenQual',
 'FireplaceQu',
 'GarageQual',
 'GarageCond',
 'PoolQC']

### Import Data

In [86]:
data_df = pd.read_csv("datasets\housing-price\iter-7\housing_prices.csv")

In [87]:
data_df.duplicated().sum()

0

In [88]:
data_df.dtypes.value_counts()

object     43
int64      35
float64     3
dtype: int64

In [89]:
data_df.SalePrice

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [93]:
data_df.corrwith(data_df.SalePrice).sort_values()

KitchenAbvGr    -0.135907
EnclosedPorch   -0.128578
MSSubClass      -0.084284
OverallCond     -0.077856
YrSold          -0.028923
LowQualFinSF    -0.025606
Id              -0.021917
MiscVal         -0.021190
BsmtHalfBath    -0.016844
BsmtFinSF2      -0.011378
3SsnPorch        0.044584
MoSold           0.046432
PoolArea         0.092404
ScreenPorch      0.111447
BedroomAbvGr     0.168213
BsmtUnfSF        0.214479
BsmtFullBath     0.227122
LotArea          0.263843
HalfBath         0.284108
OpenPorchSF      0.315856
2ndFlrSF         0.319334
WoodDeckSF       0.324413
LotFrontage      0.351799
BsmtFinSF1       0.386420
Fireplaces       0.466929
MasVnrArea       0.477493
GarageYrBlt      0.486362
YearRemodAdd     0.507101
YearBuilt        0.522897
TotRmsAbvGrd     0.533723
FullBath         0.560664
1stFlrSF         0.605852
TotalBsmtSF      0.613581
GarageArea       0.623431
GarageCars       0.640409
GrLivArea        0.708624
OverallQual      0.790982
SalePrice        1.000000
dtype: float

In [62]:
data_df = prepare_data(data_df)

In [63]:
data_df.shape

(1460, 75)

## Split

In [64]:
y = data_df.pop("SalePrice")
X = data_df.copy()

X_train,X_test,y_train,y_test = train_test_split(X,y
                                                 ,test_size=0.2
                                                 ,random_state=555
                                                 )

## Pipeline

In [65]:
all_cat_cols = list(X_train.select_dtypes(exclude="number"))
qu_cats = find_columns(all_cat_cols,which="quality")
X_cat_qu_col = qu_cats[0]
X_cat_qu_list = qu_cats[1]

In [66]:
noqu_cats = find_columns(all_cat_cols,which="notquality")
X_cat_noqu_col = noqu_cats[0]
category_array=[]
for i in range(len(X_cat_qu_col)):
    category_array.append(X_cat_qu_list)

In [67]:
qu_cats = find_columns(all_cat_cols,which="ordinal")
X_cat_ord_col = qu_cats[0]
X_cat_ord_list = qu_cats[1]

In [77]:
impute_num = SimpleImputer(strategy="mean")

impute_cat = SimpleImputer(strategy="constant",fill_value="NA")
encode_cat = OneHotEncoder(drop="first",sparse_output = False,handle_unknown="infrequent_if_exist",min_frequency=6)
encode_cat_qu = OrdinalEncoder(categories=category_array)
encode_cat_ord = OrdinalEncoder(categories=X_cat_ord_list)

scale = StandardScaler()
model = DecisionTreeRegressor(max_depth=11,min_samples_leaf=5,min_samples_split=31)

split_cats = ColumnTransformer(transformers=
                                [("encode_cat_qu", encode_cat_qu, X_cat_qu_col),
                                 ("encode_cat", encode_cat, X_cat_noqu_col),
                                 ("encode_cat_ord", encode_cat_ord, X_cat_ord_col)])

pipe_cat_all = Pipeline(steps=[("impute_cat",impute_cat),("split_cats",split_cats)])
pipe_num = Pipeline(steps=[("impute_num",impute_num)])

split_num_cat = ColumnTransformer(transformers=
                                [("pipe_num", pipe_num, make_column_selector(dtype_include="number")),
                                 ("pipe_cat_all", pipe_cat_all, make_column_selector(dtype_exclude="number"))])


pipe_all = Pipeline(steps=[("split_num_cat",split_num_cat)
                           #,("pca",pca)
                           ,("model",model)]).set_output(transform="pandas")

In [78]:
pipe_all.fit(X_train,y_train)

Pipeline(steps=[('split_num_cat',
                 ColumnTransformer(transformers=[('pipe_num',
                                                  Pipeline(steps=[('impute_num',
                                                                   SimpleImputer())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000015D9ECD7D30>),
                                                 ('pipe_cat_all',
                                                  Pipeline(steps=[('impute_cat',
                                                                   SimpleImputer(fill_value='NA',
                                                                                 strategy='constant')),
                                                                  ('split_cats',
                                                                   ColumnTransformer(transfor...
                                                                                                                                'RFn',
                                                                                                                                'Fin'],
                                                                                                                               ['NA',
                                                                                                                                'N',
                                                                                                                                'P',
                                                                                                                                'Y']]),
                                                                                                    ['Utilities',
                                                                                                     'LandSlope',
                                                                                                     'RoofMatl',
                                                                                                     'MasVnrType',
                                                                                                     'Foundation',
                                                                                                     'BsmtExposure',
                                                                                                     'BsmtFinType1',
                                                                                                     'BsmtFinType2',
                                                                                                     'GarageFinish',
                                                                                                     'PavedDrive'])]))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000015D9ECD7FA0>)])),
                ('model',
                 DecisionTreeRegressor(max_depth=11, min_samples_leaf=5,
                                       min_samples_split=31))])

#### Evaluate

In [79]:
y_train_pred = pipe_all.predict(X_train)

train_mae  = mean_absolute_error(y_train, y_train_pred)
train_mse  = mean_squared_error(y_train, y_train_pred)
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

In [80]:
y_test_pred = pipe_all.predict(X_test)

test_mae  = mean_absolute_error(y_test, y_test_pred)
test_mse  = mean_squared_error(y_test, y_test_pred)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_mape = mean_absolute_percentage_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

c:\Python39\lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [12] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [81]:
comparing_models_df = pd.DataFrame({"MAE":  [train_mae,  test_mae ],
                                    "RMSE": [train_rmse, test_rmse],
                                    "MAPE": [train_mape, test_mape],
                                    "R2":   [train_r2,   test_r2  ]},
                                    index=["train_set", "test_set"])
comparing_models_df

,MAE,RMSE,MAPE,R2
train_set,17848.778830,27288.005928,0.101020,0.883032
test_set,22573.699925,34987.804631,0.126474,0.798159
